In [2]:
from constant import NEW_LOG, OLD_LOG
from data_check import get_trace
from json_processor import CompareBuild 
import json
import os
os.chdir('/Users/pkun/PycharmProjects/ui_api_automated_test')

In [3]:
new_trace = get_trace(NEW_LOG) 
old_trace = get_trace(OLD_LOG)

In [4]:
# api_index = 0 if log == OLD_LOG else 3
new_trace_set = set()
old_trace_set = set() 
get_trace_from_traces = lambda traces, y : set( j[y] for i in traces for j in traces[i])
new_trace_set = get_trace_from_traces(new_trace, 3)
old_trace_set = get_trace_from_traces(old_trace, 0)
join_set = new_trace_set.intersection(old_trace_set) 
join_set

{'add_your_bank_card_to_submit_your_bank_card_number',
 'change_reserved_mobile_no_modify_reserved_mobile_no',
 'confirm_failed_message_forgot_password',
 'confirm_payment_information_to_confirm_payment_and_require_wechat_user_to_submit_payment_password_authorization',
 'confirm_payment_result_and_confirm_completion',
 'select_short_mobile_phone_number_to_update_bank_card_to_reserve_mobile_phone_number',
 'select_short_phone_number_to_add_mobile_phone_number_by_binding_new_card',
 'select_the_short_phone_number_to_select_the_mobile_number',
 'submit_bank_card_and_identity_information_submit_bank_card_and_identity_information',
 'submit_bank_card_and_identity_information_to_view_the_user_agreement_and_do_not_need_to_show_the_list_of_protocols',
 'submit_payment_password_authorization_payment_authorization_and_need_wechat_user_verification_short',
 'submit_payment_password_authorize_payment_and_verify_payment_password_is_illegal',
 'submit_payment_password_authorized_payment_view_offer_l

In [5]:
join_set.remove('system_requests_from_direct_business')

In [6]:
def get_file_index(file_with_trace, choose_):
    file_index = dict()
    for file in file_with_trace:
        for traces in file_with_trace[file]:
            for trace in traces:
                if trace in join_set:
                    l = file_index.setdefault(trace, set())
                    l.add(file)
    # for trace in file_index:
    #     copy_set = file_index[trace].copy()
    #     for file in copy_set:
    #         j = json.load(open(file, 'r'))
    #         if choose_(j['success']):
    #             file_index[trace].remove(file)
    return file_index

In [7]:
# file_index is a dict which contains all false test cases which contains trace in new_trace
new_false_file_index = get_file_index(new_trace, lambda x : x)
old_true_file_index = get_file_index(old_trace, lambda x : not x)

In [8]:
def get_all_ui_info(file_index, index):
    join_ret = dict()
    for trace in file_index:
        join_ret[trace] = list() 
        for file in file_index[trace]:
            cb = CompareBuild(file, index)
            res = cb.json_file_process()
            for i in res:
                if i['query'] != trace.replace('_', ' '):
                    continue
                else:
                    i.pop('query')
                    join_ret[trace].append(i)
    return join_ret

In [9]:
join_ret_of_new = get_all_ui_info(new_false_file_index, True)
join_ret_of_old = get_all_ui_info(old_true_file_index, False)

In [10]:
def show_head_of_join_ret(join_ret):
    for key_trace in join_ret.keys():
        for res in join_ret[key_trace]:
            print(res)
            break
        break


In [11]:
show_head_of_join_ret(join_ret_of_new)
show_head_of_join_ret(join_ret_of_old)

{'positive_doc': {'text': '立即支付', 'content-desc': '立即支付', 'resource-id': 'com.tencent.mm:id/confirm_pay_btn', 'class': 'android.widget.TextView'}, 'negative_docs': [{'content-desc': '当前所在页面,支付'}, {'content-desc': '返回'}, {'text': '支付'}, {'text': 'body'}, {'text': '￥0.05'}, {'text': '收款方'}, {'text': 'UAT普通直连测试商户'}, {'text': '支付安全由中国人民财产保险股份有限公司承保'}]}
{'positive_doc': {'text': '立即支付', 'content-desc': '立即支付', 'resource-id': 'com.tencent.mm:id/confirm_pay_btn', 'class': 'android.widget.TextView'}, 'negative_docs': [{'content-desc': '当前所在页面,支付'}, {'text': '支付'}, {'content-desc': '返回'}, {'text': 'body'}, {'text': '￥0.05'}, {'text': '收款方'}, {'text': 'UAT普通直连测试商户'}, {'text': '支付安全由中国人民财产保险股份有限公司承保'}]}


上面的流程是从新老数据集中取出数据，格式为
```json
{
    "filePath":["all trace"]
}
```
然后从所有trace中取出query，然后在所有的query，也就是api上做交集，得出一个新的api集合join_set。然后从所有数据中筛出存在join set的文件，file_index 就是新老数据集中所有包含了join_set中api的文件，get_all_ui_info是取出了所有file_index里的数据，数据格式和json_processor里基本上一样。
下面要做的事情是，从这里面比较新旧数据集同api下，描述的不同

In [ ]:
def compare_two_ui_info(new_ui_info, old_ui_info):
    n_class = new_ui_info['class']
    n_resource_id = new_ui_info['resource-id']
    n_text = new_ui_info['text']
    n_content_desc= new_ui_info['content-desc']
    o_class = old_ui_info['class']
    o_resource_id = old_ui_info['resource-id']
    o_text = old_ui_info['text']
    o_content_desc= old_ui_info['content-desc']

In [19]:
count = 0
for key_trace in join_set:
    print(key_trace)
    for i in join_ret_of_new[key_trace]:
        for j in join_ret_of_old[key_trace]:
            print(i['positive_doc'])
            print(j['positive_doc'])
            print()
        print()
    count += 1
    if count == 5:
        break

submit_your_bank_card_and_identity_information_cancel_your_bank_card_and_identity_information
{'content-desc': '返回', 'resource-id': 'com.tencent.mm:id/actionbar_up_indicator_btn', 'class': 'android.widget.ImageView'}
{'content-desc': '返回', 'resource-id': 'com.tencent.mm:id/actionbar_up_indicator_btn', 'class': 'android.widget.ImageView'}

{'content-desc': '返回', 'resource-id': 'com.tencent.mm:id/actionbar_up_indicator_btn', 'class': 'android.widget.ImageView'}
{'content-desc': '返回', 'resource-id': 'com.tencent.mm:id/actionbar_up_indicator_btn', 'class': 'android.widget.ImageView'}

{'content-desc': '返回', 'resource-id': 'com.tencent.mm:id/actionbar_up_indicator_btn', 'class': 'android.widget.ImageView'}
{'content-desc': '返回', 'resource-id': 'com.tencent.mm:id/actionbar_up_indicator_btn', 'class': 'android.widget.ImageView'}


add_your_bank_card_to_submit_your_bank_card_number
{'resource-id': 'com.tencent.mm:id/tenpay_push_down', 'class': 'android.widget.RelativeLayout'}
{'resource-id': '